In [ ]:
import re, pandas as pd, datetime, decimal, bs4
def toDecimalConversion(number,quant=20):
    quant = decimal.Decimal(str(quant))
    quant = 1*10**(-quant)
    decimal_return = decimal.Decimal(str(number)).quantize(quant)
    return decimal_return

In [ ]:
with open('dataimport1.html','r',encoding='utf-8') as file:
    soup = bs4.BeautifulSoup(file.read(),'html.parser')
    with open('dataimport1a.html','w',encoding='utf-8') as nfile:
        nfile.write((soup.prettify()))

In [ ]:
with open('dataimport1a.html','r') as file:
    data = file.read()
extractDF = pd.DataFrame(columns=['Date','Time','Actual','Consensus'])

In [ ]:
info = re.findall('<tr.*?historicEvent.*?</tr>',data,re.DOTALL)
for i in info:
    prelim = re.search('Preliminary Release',i,re.DOTALL)
    month_re = re.search('<td.*?\n(.*?)\n.*?</td>',i,re.DOTALL)
    time_re = re.search('.*?</td>.*?<td.*?\n(.*?)\n.*?</td>',i,re.DOTALL)
    actual_re = re.search('.*?</td>.*?</td>.*?<td.*?\n.*?\n(.*?)\n.*?</td>',i,re.DOTALL)
    consensus_re = re.search('.*?</td>.*?</td>.*?</td>.*?<td.*?\n(.*?)\n.*?</td>',i,re.DOTALL)
    actual_re  =actual_re.group(1)
    consensus_re = consensus_re.group(1)
    for x in ['M','%','K','B']:
        actual_re = actual_re.replace(x,'')
        consensus_re = consensus_re.replace(x,'')
    actual_re = toDecimalConversion(actual_re,6)
    try:
        consensus_re = toDecimalConversion(consensus_re,6)
    except:
        continue
    row = {'Date':month_re.group(1),'Time':time_re.group(1).replace(' ',''),'Actual':actual_re,'Consensus':consensus_re}
    extractDF.loc[len(extractDF)] = row

In [ ]:

extractDF['Difference'] = extractDF['Actual'] - extractDF['Consensus']

extractDF['Actual'] = extractDF['Actual'].apply(pd.to_numeric)

extractDF['Consensus'] = pd.to_numeric(extractDF['Consensus'],errors='coerce')

In [ ]:
dst_str = {
    2023:('2023-03-12','2023-11-05'),
    2022:('2022-03-13','2022-11-06'),
    2021:('2021-03-14','2021-11-07'),
    2020:('2020-03-08','2020-11-01'),
    2019:('2019-03-10','2019-11-03'),
    2018:('2018-03-11','2018-11-04'),
    2017:('2017-03-12','2017-11-05'),
    2016:('2016-03-13','2016-11-06'),
    2015:('2015-03-08','2015-11-01'),
    2014:('2014-03-09','2014-11-02'),
    2013:('2013-03-10','2013-11-03'),
    2012:('2012-03-11','2012-11-04'),
    2011:('2011-03-13','2011-11-06'),
    2010:('2010-03-14','2010-11-07'),
    2009:('2009-03-08','2009-11-01'),
    2008:('2008-03-09','2008-11-02'),
    }
dst = {}
for i in dst_str:
    temptup = []
    for j in dst_str[i]:
        j = pd.to_datetime(j)
        temptup.append(j)
    dst[i] = temptup

In [ ]:
extractDFDates = extractDF.copy()
for i,r in extractDF.iterrows():
    if '(' in r["Date"]:
        date = re.search('(.*?)\(.*',r['Date'])
        newdate = date.group(1).strip()
    else:
        newdate = r['Date']
    
    newdate = f'{newdate} {r["Time"]}'
    newdate = pd.to_datetime(newdate)
    if newdate>dst[newdate.year][0] and newdate<dst[newdate.year][1]:
        newdate = newdate + datetime.timedelta(hours=8)
    else:
        newdate = newdate + datetime.timedelta(hours=7)
    extractDFDates['Date'].at[i] = newdate
extractDFDates.drop(columns=['Time'],inplace=True) 

In [ ]:
extractDFDates.to_csv('extractDF.csv')